In [1]:
import shutil
import os
import base64
from pdf2image import convert_from_path
from PIL import Image
import io
import requests
import json
import pandas as pd

In [8]:
with open('../../key.txt', 'r') as file:
    secret_key = file.read().strip()
os.environ['OPENAI_API_KEY'] = secret_key

In [9]:
def convert_pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)[0]

In [10]:
def encode_image_to_base64(image):
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

In [11]:
def get_openai_response(base64_image,prompt):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
    }
    
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 400
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    if response.status_code == 200:
        response_json = response.json()

        raw_json = response_json['choices'][0]['message']['content']
        response_text = raw_json.replace("json", "").replace("```", "").strip()
        # print(response_text)
        try:
            response_data = json.loads(response_text)

            invoice_date = response_data.get('invoice_date')
            currency = response_data.get('currency')
            po_number = response_data.get('po_number')
            
            suspended_tax_amount = response_data.get('suspended_tax_amount')
            vat_amount = response_data.get('vat_amount')
            delivery_note_number = response_data.get('delivery_note_number')
            
            invoice_amount = response_data.get('invoice_amount')
            invoice_no = response_data.get('invoice_no')
            sub_total = response_data.get('sub_total')
            
        except json.JSONDecodeError as e:
            print(e)
            (
                invoice_date, currency, po_number,
                suspended_tax_amount, vat_amount, delivery_note_number,
                invoice_amount, invoice_no, sub_total,
            )  = (
                None, None, None, 
                None, None, None, 
                None, None, None, 
            )
        return (
                invoice_date, currency, po_number,
                suspended_tax_amount, vat_amount, delivery_note_number,
                invoice_amount, invoice_no, sub_total,
            )
    else:
        print(f"Error in OpenAI API response: {response.status_code} - {response.text}")
        return (
                None, None, None, 
                None, None, None, 
                None, None, None, 
            )

In [12]:
def convert_string_to_amount(value):
    try:
        converted_value = value.replace('\s', '').replace(',', '')
        return float(converted_value)
    except:
        return 0

In [129]:
def get_sbu(mapping_dataset, value):
    _filtered_data = mapping_dataset[(mapping_dataset['min']<=value) & (mapping_dataset['max']>=value)]
    if len(_filtered_data)>=1:
        return list(_filtered_data['sbu'])[0]
    else:
        return None

In [95]:
def convert_po_number_to_int(value):
    return int(value)

In [121]:
def convert_po_num_to_list(po_num, sbu_mapping_table):
    validated_po_num = []
    cleaned_po_num = []
    sbu = None
    if (po_num == "null") or (po_num == "") or (po_num is None):
        po_num = [""]
    
    if type(po_num) is not list:
        po_num = [po_num]

    for i in po_num:
        _po_num = i.replace('O', '0').replace('o', '0')
        cleaned_po_num.append(_po_num)
        if len(_po_num)>=8:
            try:
                _po_num = convert_po_number_to_int(_po_num)
                _sbu = get_sbu(sbu_mapping_table, _po_num)
                if _sbu is not None:
                    validated_po_num.append(_po_num)
                    sbu = _sbu
                else:
                    validated_po_num.append('wrong po incorrect sbu')
            except Exception as e:
                print(e)
                validated_po_num.append('wrong po non_integer')
        else: 
            validated_po_num.append('wrong po')

    return validated_po_num, cleaned_po_num, sbu

In [75]:
def create_json_output(path_pdf, prompt, sbu_mapping_table):
    image_load = convert_pdf_to_images(path_pdf)
    base64_image = encode_image_to_base64(image_load)
    (
        invoice_date, currency, po_number,
        suspended_tax_amount, vat_amount, delivery_note_number,
        invoice_amount, invoice_no, sub_total,
    ) = get_openai_response(base64_image,prompt)

    suspended_tax_amount = convert_string_to_amount(suspended_tax_amount)
    vat_amount = convert_string_to_amount(vat_amount)
    invoice_amount = convert_string_to_amount(invoice_amount)
    sub_total = convert_string_to_amount(sub_total)

    overall_tax = 0
    
    invoice_type = 'Non-Tax Invoice'
    if suspended_tax_amount > 0:
        invoice_type = 'SVAT Invoice'
        overall_tax = suspended_tax_amount
    elif vat_amount > 0:
        invoice_type = 'Tax Invoice'
        overall_tax = vat_amount

    validated_po_number, cleaned_po_num, sbu = convert_po_num_to_list(po_number, sbu_mapping_table)
    
    if invoice_no or po_number  or invoice_date or invoice_amount or delivery_note_number:
        desired_output = {
            "invoice_date": invoice_date,
            "currency": currency,
            "po_number": cleaned_po_num,
            "validate_po_number": validated_po_number,
            
            "suspended_tax_amount": suspended_tax_amount,
            "vat_amount": vat_amount,
            "tax_amount": overall_tax,
            "delivery_note_number": delivery_note_number,
            
            "invoice_amount": invoice_amount,            
            "invoice_no": invoice_no,
            "sub_total": sub_total,
            "invoice_type": invoice_type,
            
            "sbu": sbu
        }
        formatted_json = json.dumps(desired_output, indent=4)
        return desired_output

In [76]:
prompt = '''Extract the following details from the invoice:
        Invoice Date,
        Currency Type,
        PO Number,
        Suspended Tax Amount,
        Vat Amount,
        Delivery Note Number Validation,
        Invoice Amount,
        Invoice Number,
        Sub Total
        from the image and return the response as a JSON object with
        'invoice_date'
        'currency',
        'po_number',
        'suspended_tax_amount',
        'vat_amount',
        'delivery_note_number',
        'invoice_amount',
        'invoice_no',
        'sub_total',
         as keys.

    must return json object only no more any text provide

    1. Invoice Date
        Extract the accurate Invoice Date from the invoice.
    
        Consider alternative names such as:
            Date
            Invoice Date
    
        Invoice Date that could be in various formats such as: 
            10/24/2024, 
            Oct/24/2024, 
            24/10/2024, 
            24/Oct/2024.             
        convert it to a uniform format DD/MM/YYYY.
    
    2. Currency Type
        I need to extract the standard currency type used in these invoices. Here's the process to follow:
        1. **Direct Currency Extraction**:
           - Search for common currency symbols (e.g., $, €, £) or currency codes (e.g., USD, EUR, GBP) within the text of the PDF.
           - If a currency is mentioned, extract it as the standard currency type.

        2. **Currency Inference from Country**:
           - If no currency is mentioned in the invoice, check for the presence of a country name in the address section.
           - Use the country name to infer the currency type. For example, if the country is "United States," the currency should be USD; if the country is "Germany," the currency should be EUR, etc.

        3. **Default Currency**:
           - If neither currency nor country is mentioned in the invoice, default the standard currency type to LKR (Sri Lankan Rupee).

        Consider the following example country-to-currency mappings:
            - United States -> USD
            - Eurozone countries (Germany, France, etc.) -> EUR
            - United Kingdom -> GBP
            - Australia -> AUD
            - Canada -> CAD
            - India -> INR
            - Japan -> JPY
            - Default -> LKR
        
        Also, include any additional steps or checks that might be useful for ensuring accurate extraction and inference.

    3. PO Number
    
        There could be multiple "PO Numbers".
        
        3.1 In the invoices, "PO Number" can be mentioned using various alternative names:
            "PO number"
            "Purchase order number"
            "Order Number (NO)"
            "Buyer order number"
            "Your order reference number"
            "PO No"
            "Customer PO"
            "Cust. PO No"
            "Manual No"
            "Order Ref"
        
        3.2 Once a "PO Number" is extracted, it should be validated based on the following criteria:
            3a) It must contain 8 to 10 characters.
            3b) All characters of the "PO Number" should be digits.
            If the extracted "PO Number" does not meet criteria 3a or 3b, it should be flagged as "wrong PO number".
        
        3.3 Additionally, there are edge cases where the "PO Number" might have more than 10 characters, such as:
            * 2341234562(1748)
            * 2341234562_1748
            * 2341234562 1748   
            
            In these cases, "PO Number" is only upto the special character (i.e., "(", "_", or " "). 
            In this example, the final output should be ["2341234562"]. 
        
        Requirement:
            Identifies all "PO Numbers" based on the alternative names.
            Check for the edge cases.
            Validates each extracted "PO Number" based on the criteria mentioned above.
            Data type of the "Invoice Amount" is str.

    4. Suspended Tax Amount
        Need to extract the "Suspended Vat" amount. 
        Note that the "Suspended Vat" can also be referred to as "SVAT" or "Suspended Tax". 
        
        Follow these conditions:
            The "Suspended Vat" amount should be a numerical value, which can include decimal points.
            If the invoice does not have a "Suspended Vat" amount, the default value should be 0.
            
        Additional Instructions:
            Ensure to search for all possible synonyms ("SVAT", "Suspended Vat", "Suspended Tax").
            Accurately extract the numerical value associated with these terms.
            If none of these terms are found or if no numerical value is associated with them, set the "suspended_vat_amount" to 0.
            Data type of the "Invoice Amount" is str.

    5. Vat Amount
        Need to extract the "Vat" amount. Note that the "Vat" can also be referred to as "Tax". 
        
        Follow these conditions:
            The "Vat" amount should be a numerical value, which can include decimal points.
            If the invoice does not have a "Vat" amount, the default value should be 0.

        Additional Instructions:
            Ensure to search for all possible synonyms ("Vat", "Tax").
            Accurately extract the numerical value associated with these terms.
            If none of these terms are found or if no numerical value is associated with them, set the "vat_amount" to 0.
            Data type of the "Invoice Amount" is str.

    6. Invoice Amount
        Need to extract the "Invoice Amount" from the invoice. 
        The "Invoice Amount" can be referred to using different terms, including but not limited to:
            Invoice Amount
            Invoice Value
            Grand Total
            Total Amount
            Invoice Total
            Gross Value
        
        Please follow these steps:
        
            Identify the section of the invoice where the total amount is listed. Look for the following synonyms to locate this section:
                Invoice Amount
                Invoice Value
                Grand Total
                Total Amount
                Invoice Total
                Gross Value
        
            Extract the numerical value associated with the identified term. 
            
            The value should be a number and can include decimal points.

            Check the extracted numerical value for formatting issues such as improperly placed decimal points. 
                e.g., "2145.046.40" should be interpreted as "2145046.40"
                 Assume the rightmost part after the last decimal point is the actual decimal part.

            Data type of the "Invoice Amount" is str.

    7. Sub Total
        Need to extract the "Sub Total" from the invoice. 
        The "Sub Total" can be referred to using different terms, including but not limited to:
            Sub Total
            Sub Amount
            Net Total
            Net Amount
        
        Please follow these steps:
            Identify the section of the invoice where the sub total amount is listed. Look for the following synonyms to locate this section:
                Sub Total
                Sub Amount
                Net Total
                Net Amount
                
            Extract the numerical value associated with the identified term. 
            
            The value should be a number and can include decimal points.
            
            Ensure that the extracted value is the intermediate total amount before any taxes, discounts, or additional charges are applied, not the final total amount due on the invoice.

            Check the extracted numerical value for formatting issues such as improperly placed decimal points. 
                e.g., "2145.046.40" should be interpreted as "2145046.40". 
                 Assume the rightmost part after the last decimal point is the actual decimal part.
                        
            Output the extracted numerical value as the "Sub Total."

            Data type of the "Invoice Amount" is str.

    8. Delivery Note Number Validation
        when considering the 'Delivery note number' should follow below details:
        Extract the accurate Delivery note number from the invoice.
            Consider alternative names such as:
                Delivery note number
                Dispatch note number
                DO Number
                AOD
                Delivery order Number
                DN Number
                Advise No
                Delivery note number ≠ Job No
            
     9. Invoice Number
        'Invoice No' Consider alternative names such as:
            Invoice No
            Invoice number	
            Invoice Ref
            Invoice Reference
            Our reference No

General Rules
If any value is missing, set it to null only.
    '''

In [124]:
base_location = "../../"
sbu_mapping = (
    pd.read_csv(os.path.join(base_location,'conf', 'sbu_type.csv'))
)
sbu_mapping[['min', 'max']] = sbu_mapping[['min', 'max']].apply(pd.to_numeric)

In [130]:
# base_folder = "upload\\date inconsistancy"
# base_folder = "upload\\Other currencies"
# base_folder = "upload\\prefix suffix"
# base_folder = "upload\\invoice_type_consistancy"
# base_folder = "upload\\missing_digits_in_amount"
invoice_folder_name = 'clean poly'
base_folder = os.path.join(base_location, 'data', invoice_folder_name)
output_folder = os.path.join(base_location, 'data')

all_files = [f for f in os.listdir(base_folder) if os.path.isfile(os.path.join(base_folder, f))]
batch_files = [f for f in all_files if f.endswith(('.pdf', '.png', '.jpg', '.jpeg'))]

In [ ]:
Invoice No, Invoice Date, SBU, PO number, Delivery Note Number, Invoice Type, Sub Total, Tax Amount, Invoice Amount, Comments, File Name

In [135]:
output_dc['clean poly_10_10_2023_001 highlight.pdf'].keys()

dict_keys(['invoice_date', 'currency', 'po_number', 'validate_po_number', 'suspended_tax_amount', 'vat_amount', 'delivery_note_number', 'invoice_amount', 'invoice_no', 'sub_total', 'invoice_type', 'sbu'])

In [128]:
output_dc = {}
for i in batch_files:
    try:
        outcome = create_json_output(os.path.join(base_folder, i), prompt, sbu_mapping)
        if outcome is not None:
            output_dc[i] = outcome
        else:
            print('error : ', i)
    except:
        print('error : ', i)

select_columns = [
    'invoice_no', 'invoice_date', 'invoice_type', 'sbu', 
    'po_number', 'delivery_note_number', 'sub_total', 'tax_amount', 
    'invoice_amount']

df_abc = pd.DataFrame.from_dict(output_dc, orient='index').reset_index().rename(columns={'index': 'filename'})
df_abc[select_columns].to_excel(os.path.join(output_folder, "output {}.xlsx".format(invoice_folder_name)), index=False)

clean poly_10_10_2023_001 highlight.pdf
clean poly_10_10_2023_001.pdf
clean poly_10_10_2023_002.pdf
clean poly_10_10_2023_003.pdf
37    C600
Name: sbu, dtype: object
clean poly_10_10_2023_004.pdf
clean poly_10_10_2023_005.pdf
clean poly_10_10_2023_006.pdf
clean poly_10_10_2023_007.pdf
invalid literal for int() with base 10: 'GL2002825'
clean poly_10_10_2023_008.pdf
37    C600
Name: sbu, dtype: object
clean poly_10_10_2023_009.pdf


C100Febuary15Prima006_1.pdf
C100Jan10JF & I006.pdf
C100Jan11Airline Trade004.pdf
C200Akhtari Trades003.pdf
C200Akhtari Trades003_1.pdf
C200DPO Lanka001 (1).pdf
C200DPO Lanka001.pdf
C200JF&I Packaging004.pdf
C200JF&I Packaging028.pdf
C200Print Care001.pdf
C200Serandib Flour Mills007.pdf
C200Serandib Flour Mills016.pdf
C200United Tractor001.pdf
Tars lanka016.pdf


In [122]:
# base_folder = "upload\\date inconsistancy"
# base_folder = "upload\\Other currencies"
# base_folder    "upload\\prefix suffix"
# base_folder = "upload\\invoice_type_consistancy"
# base_folder = "upload\\missing_digits_in_amount"
base_folder = os.path.join(base_location, 'data', 'clean poly')

path_pdf = os.path.join(base_folder, 'clean poly_10_10_2023_001 highlight.pdf')
create_json_output(path_pdf, prompt, sbu_mapping)

{'invoice_date': '06/02/2023',
 'currency': 'USD',
 'po_number': ['6100226252'],
 'validate_po_number': ['wrong po incorrect sbu'],
 'suspended_tax_amount': 0,
 'vat_amount': 0,
 'delivery_note_number': None,
 'invoice_amount': 807.3,
 'invoice_no': 'FS-00211',
 'sub_total': 807.3,
 'invoice_type': 'Non-Tax Invoice',
 'sbu': None}

In [137]:
base_folder

'../../data\\clean poly'

In [136]:
# base_folder = "upload\\date inconsistancy"
# base_folder = "upload\\Other currencies"
# base_folder    "upload\\prefix suffix"
# base_folder = "upload\\invoice_type_consistancy"
# base_folder = "upload\\missing_digits_in_amount"
base_folder = os.path.join(base_location, 'data', 'clean poly')

path_pdf = os.path.join(base_folder, 'clean poly_10_10_2023_001 highlight.pdf')
create_json_output(path_pdf, prompt, sbu_mapping)

PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file '../../data\clean poly\clean poly_10_10_2023_001 highlight.pdf': No error.


In [108]:
# base_folder = "upload\\date inconsistancy"
# base_folder = "upload\\Other currencies"
# base_folder    "upload\\prefix suffix"
# base_folder = "upload\\invoice_type_consistancy"
# base_folder = "upload\\missing_digits_in_amount"
base_folder = os.path.join(base_location, 'data', 'clean poly')

path_pdf = os.path.join(base_folder, 'clean poly_10_10_2023_001 highlight.pdf')
create_json_output(path_pdf, prompt, sbu_mapping)

     sbu  doc   description        min        max
37  C600  Z60  NFL-LOCAL PO  610000000  619999999


KeyError: 0